# Day 5-6교시: 스코프 + 데코레이터 개념

**학습 시간**: 50분  
**학습 목표**:

- 지역 변수와 전역 변수의 차이를 이해하고 적절히 활용하기
- 클래스 변수와 인스턴스 변수를 구분하여 사용하기
- 데코레이터(@) 문법의 개념과 동작 원리 이해하기
- FastAPI에서 @app.get() 등의 데코레이터 활용 미리보기
- @property 데코레이터로 메서드를 속성처럼 사용하기

---

## 📖 스코프(Scope)란 무엇인가?

### 변수의 "유효 범위"를 의미합니다

여러분이 집에서 물건을 찾는다고 상상해보세요. 어떤 물건은 내 방에만 있고, 어떤 물건은 거실에 있어서 온 가족이 사용하죠. 프로그래밍에서도 마찬가지입니다!

**스코프(Scope)**는 변수가 "어디서 접근 가능한지"를 정의하는 규칙입니다:
- **지역 변수(Local Variable)**: 함수 안에서만 사용 가능 (내 방의 물건)
- **전역 변수(Global Variable)**: 프로그램 어디서나 사용 가능 (거실의 물건)

### 왜 스코프를 이해해야 할까요?

1. **코드의 안전성**: 함수 내부의 변수가 외부에 영향을 주지 않도록
2. **메모리 효율**: 필요한 곳에서만 변수가 존재하도록
3. **디버깅 용이**: 변수 이름이 겹쳐도 문제없이 동작
4. **FastAPI 실무**: 설정값, 카운터 등 전역 상태 관리에 필수

### 실무에서는 어떻게 사용될까요?

FastAPI로 웹 서버를 만들 때:
```python
# 전역 변수로 설정 관리
DATABASE_URL = "sqlite:///./app.db"  # 모든 함수에서 사용

@app.get("/users")
def get_users():
    limit = 10  # 지역 변수: 이 함수에서만 사용
    # DATABASE_URL은 접근 가능! (전역)
    # 다른 함수의 limit는 접근 불가! (지역)
```

### ⚠️ 주의할 점

- 전역 변수를 너무 많이 사용하면 코드가 복잡해집니다
- 함수 내부에서 전역 변수를 수정할 때는 `global` 키워드 필수
- 가능하면 지역 변수를 사용하고, 필요한 값은 매개변수로 전달하세요

---

## 1. 지역 변수(Local Variable)

### 함수 안에서만 존재하는 변수

지역 변수는 **함수 내부에서 선언되고, 함수가 끝나면 사라지는** 변수입니다. 마치 함수라는 방 안에서만 사용할 수 있는 물건과 같습니다.

**핵심 특징**:
- 함수 내부에서만 접근 가능
- 함수 실행이 끝나면 메모리에서 자동 삭제
- 다른 함수의 같은 이름 변수와 충돌하지 않음

**왜 지역 변수를 사용할까요?**
- **독립성**: 각 함수가 독립적으로 동작
- **안전성**: 실수로 다른 곳의 값을 바꿀 걱정 없음
- **메모리 효율**: 필요할 때만 생성되고 자동으로 삭제


In [ ]:
# 지역 변수 예제
def calculate_area():
    width = 10  # 지역 변수: 이 함수 안에서만 사용 가능
    height = 5  # 지역 변수: 이 함수 안에서만 사용 가능
    area = width * height  # 지역 변수끼리 계산
    return area  # 계산 결과만 반환

# 함수 실행
result = calculate_area()
print("면적:", result)  # 출력: 면적: 50

# 아래 코드는 에러 발생!
# print(width)  # NameError: name 'width' is not defined
# width는 함수 안에서만 존재하므로 밖에서는 접근 불가!

**실행 결과:**
```
면적: 50
```

**코드 해석:**
1. `calculate_area()` 함수가 실행되면 `width`, `height`, `area` 변수가 생성됩니다
2. 계산을 마치고 `area` 값(50)을 반환합니다
3. 함수 실행이 끝나면 `width`, `height`, `area`는 메모리에서 사라집니다
4. 함수 밖에서 `width`에 접근하려고 하면 "그런 변수 없다"는 에러가 발생합니다

### 💡 알아두기

- 지역 변수는 **함수가 호출될 때마다 새로 생성**됩니다
- 같은 함수를 여러 번 호출해도, 각 호출은 독립적인 지역 변수를 가집니다
- 이것이 바로 재사용 가능한 함수를 만들 수 있는 이유입니다!

---

## 2. 전역 변수(Global Variable)

### 프로그램 어디서나 접근 가능한 변수

전역 변수는 **함수 밖에서 선언되어, 프로그램 전체에서 사용할 수 있는** 변수입니다. 마치 거실에 있는 TV처럼 집 안 누구나 사용할 수 있는 물건과 같습니다.

**핵심 특징**:
- 프로그램 시작부터 끝까지 존재
- 모든 함수에서 읽기 가능
- 수정하려면 `global` 키워드 필요

**실무 활용 사례**:
- 설정값 저장 (DATABASE_URL, SECRET_KEY)
- 전체 통계 추적 (방문자 수, 요청 횟수)
- 공유 자원 관리 (데이터베이스 연결)


In [ ]:
# 전역 변수 예제
total_count = 0  # 전역 변수: 프로그램 전체에서 사용 가능

def add_count():
    global total_count  # ⭐ global 키워드로 전역 변수 수정 선언
    total_count += 1  # 전역 변수 값을 1 증가
    print(f"현재 카운트: {total_count}")

# 함수를 여러 번 호출
add_count()  # 출력: 현재 카운트: 1
add_count()  # 출력: 현재 카운트: 2
add_count()  # 출력: 현재 카운트: 3

# 함수 밖에서도 total_count 접근 가능!
print(f"최종 카운트: {total_count}")  # 출력: 최종 카운트: 3

**실행 결과:**
```
현재 카운트: 1
현재 카운트: 2
현재 카운트: 3
최종 카운트: 3
```

**코드 해석:**
1. `total_count`는 함수 밖에서 선언되어 전역 변수입니다
2. `global total_count`를 선언해야 함수 내부에서 값을 수정할 수 있습니다
3. `global` 키워드가 없으면 새로운 지역 변수가 만들어져 전역 변수는 그대로입니다
4. 함수 호출이 끝나도 `total_count`는 계속 존재하며 값이 유지됩니다

### ⚠️ global 키워드를 잊으면?

```python
count = 0

def wrong_add():
    # global을 선언하지 않음!
    count = count + 1  # 에러! 지역 변수를 읽기 전에 사용
```

→ `UnboundLocalError` 발생! 함수 내부에서 전역 변수를 수정하려면 **반드시 `global` 선언**이 필요합니다.

### 💡 실무 팁

FastAPI에서 전역 설정 관리:
```python
# 전역 설정
MAX_ITEMS_PER_PAGE = 50
API_VERSION = "v1"

@app.get("/items")
def get_items(limit: int = 10):
    # 전역 변수 읽기 (global 키워드 불필요)
    if limit > MAX_ITEMS_PER_PAGE:
        limit = MAX_ITEMS_PER_PAGE
    return {"items": [...], "limit": limit}
```

---

## 3. 클래스 변수 vs 인스턴스 변수

### 클래스 레벨의 스코프 이해하기

클래스에서도 스코프 개념이 적용됩니다. 두 종류의 변수가 있습니다:

1. **클래스 변수(Class Variable)**: 모든 인스턴스가 공유하는 변수
   - 마치 학교 전체의 "총 학생 수" 같은 것
   - `클래스명.변수`로 접근
   - 모든 객체가 같은 값을 공유

2. **인스턴스 변수(Instance Variable)**: 각 인스턴스마다 독립적인 변수
   - 마치 각 학생의 "이름", "점수" 같은 것
   - `self.변수`로 접근
   - 객체마다 다른 값을 가짐

**실무 활용**:
- 클래스 변수: 생성된 객체 개수 추적, 공통 설정값
- 인스턴스 변수: 각 사용자/상품/주문의 고유 정보


In [ ]:
class Counter:
    # 클래스 변수: 모든 Counter 객체가 공유하는 값
    total = 0  # 전체 Counter 개수를 추적
    
    def __init__(self, name: str):
        # 인스턴스 변수: 각 Counter 객체마다 독립적인 값
        self.name = name  # 이 Counter의 이름
        self.count = 0  # 이 Counter의 카운트 (초기값 0)
        
        # 클래스 변수 접근: Counter.total
        Counter.total += 1  # Counter가 하나 생성될 때마다 증가
    
    def increment(self) -> None:
        """이 Counter의 카운트만 증가"""
        self.count += 1  # 인스턴스 변수 수정

# 여러 Counter 객체 생성
counter1 = Counter("첫번째")  # total = 1
counter2 = Counter("두번째")  # total = 2

# 각 Counter의 카운트는 독립적!
counter1.increment()  # counter1.count = 1
counter1.increment()  # counter1.count = 2
counter2.increment()  # counter2.count = 1

# 결과 출력
print(f"{counter1.name} 카운트: {counter1.count}")  # 출력: 첫번째 카운트: 2
print(f"{counter2.name} 카운트: {counter2.count}")  # 출력: 두번째 카운트: 1
print(f"전체 Counter 개수: {Counter.total}")  # 출력: 전체 Counter 개수: 2

**실행 결과:**
```
첫번째 카운트: 2
두번째 카운트: 1
전체 Counter 개수: 2
```

**코드 해석:**
1. `total`은 클래스 변수 → 모든 Counter 객체가 공유
2. `name`, `count`는 인스턴스 변수 → 각 Counter마다 독립적
3. `counter1`과 `counter2`는 각자의 `count`를 가지지만, `total`은 공유
4. `Counter.total`로 접근하면 전체 생성된 Counter 개수를 알 수 있음

### 📊 비교표

| 특징 | 클래스 변수 | 인스턴스 변수 |
|------|------------|-------------|
| 선언 위치 | 클래스 내부, 메서드 밖 | `__init__` 메서드 내부 |
| 접근 방법 | `클래스명.변수` | `self.변수` |
| 공유 여부 | 모든 인스턴스 공유 | 각 인스턴스 독립 |
| 용도 | 공통 설정, 통계 | 개별 객체 데이터 |

### 💡 실무 예시

```python
class User:
    # 클래스 변수: 모든 사용자 공통 설정
    max_login_attempts = 3
    
    def __init__(self, username: str, email: str):
        # 인스턴스 변수: 각 사용자마다 다른 정보
        self.username = username
        self.email = email
        self.login_attempts = 0  # 각 사용자의 로그인 시도 횟수
```

---

## 📖 데코레이터(Decorator)란 무엇인가?

### 함수를 "꾸며주는" 함수입니다

데코레이터는 기존 함수의 동작을 **변경하거나 확장**하는 강력한 도구입니다. 

**쉬운 비유**: 햄버거를 만든다고 생각해보세요!
- 기본 함수 = 패티 (핵심 기능)
- 데코레이터 = 빵, 야채, 소스 (추가 기능)
- `@데코레이터`를 붙이면 = 완성된 햄버거!

### @ 문법이 뭔가요?

`@`는 **데코레이터를 적용하는 특별한 문법**입니다:
```python
@데코레이터
def 함수():
    pass
```

이것은 다음과 같은 의미입니다:
```python
함수 = 데코레이터(함수)
```

### FastAPI에서 왜 @app.get("/users")를 사용할까요?

FastAPI의 핵심이 바로 데코레이터입니다!

```python
@app.get("/users")  # ← 이게 데코레이터!
def get_users():
    return {"users": ["Alice", "Bob"]}
```

**이 데코레이터가 하는 일**:
1. `get_users` 함수를 FastAPI에 등록
2. `/users` 경로로 GET 요청이 오면 이 함수 실행
3. 자동으로 JSON 응답 변환
4. 자동 문서화(/docs)에 추가

→ 데코레이터 덕분에 **단 2줄로 API 엔드포인트 완성**!

### 실무에서 데코레이터는 언제 사용하나요?

1. **로깅**: 함수 실행 시간, 입력/출력 기록
2. **인증**: 로그인 체크, 권한 확인
3. **캐싱**: 결과를 저장해서 재사용
4. **검증**: 입력값 검사
5. **라우팅**: FastAPI의 경로 연결 (가장 많이 사용!)

### ⚠️ 초보자 주의사항

- 데코레이터는 고급 개념이지만, **사용은 쉽습니다**
- FastAPI에서는 `@app.get()`, `@app.post()` 같은 기본 사용법만 알면 됩니다
- 직접 데코레이터를 만드는 것은 나중에 배워도 괜찮아요

---

## 4. 데코레이터 기본 개념

### 함수를 감싸서 기능을 추가하기

데코레이터의 핵심은 **함수를 입력받아, 새로운 기능이 추가된 함수를 반환**하는 것입니다.

**동작 원리**:
1. 원래 함수를 받아옴
2. 새로운 wrapper 함수를 만듦 (추가 기능 포함)
3. wrapper 안에서 원래 함수를 실행
4. wrapper 함수를 반환


In [ ]:
# 데코레이터 함수 정의
def my_decorator(func):
    """함수를 꾸며주는 데코레이터
    
    Args:
        func: 꾸며질 원래 함수
    
    Returns:
        wrapper: 기능이 추가된 새 함수
    """
    def wrapper():
        # 원래 함수 실행 "전"에 추가 기능
        print("=== 함수 실행 전 ===")
        
        # 원래 함수 실행
        func()
        
        # 원래 함수 실행 "후"에 추가 기능
        print("=== 함수 실행 후 ===")
    
    return wrapper  # 새로운 함수 반환

# @ 문법으로 데코레이터 적용
@my_decorator
def say_hello():
    """원래 함수: 인사만 출력"""
    print("안녕하세요!")

# 함수 호출
say_hello()
# 출력:
# === 함수 실행 전 ===
# 안녕하세요!
# === 함수 실행 후 ===

**실행 결과:**
```
=== 함수 실행 전 ===
안녕하세요!
=== 함수 실행 후 ===
```

**코드 해석:**
1. `my_decorator`는 함수(`func`)를 받아서 새 함수(`wrapper`)를 만듭니다
2. `wrapper` 안에는 "전-실행-후" 순서로 코드가 실행됩니다
3. `@my_decorator`를 붙이면 `say_hello`가 자동으로 "꾸며집니다"
4. 이제 `say_hello()`를 호출하면 실제로는 `wrapper()`가 실행됩니다

### 🔍 @ 없이 표현하면?

```python
# 이 코드는
@my_decorator
def say_hello():
    print("안녕하세요!")

# 다음과 완전히 같습니다
def say_hello():
    print("안녕하세요!")
say_hello = my_decorator(say_hello)  # 함수를 꾸며서 다시 할당
```

### 💡 실무 활용 예시

```python
# 실행 시간 측정 데코레이터
import time

def measure_time(func):
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print(f"실행 시간: {end - start:.2f}초")
    return wrapper

@measure_time
def slow_function():
    time.sleep(1)  # 1초 대기
    print("작업 완료!")
```

---

## 5. FastAPI에서의 데코레이터

### FastAPI의 핵심: 라우팅 데코레이터

FastAPI는 데코레이터를 활용해 **URL 경로와 함수를 연결**합니다. 이를 "라우팅(Routing)"이라고 합니다.

**FastAPI 라우팅 데코레이터**:
- `@app.get()`: GET 요청 처리
- `@app.post()`: POST 요청 처리
- `@app.put()`: PUT 요청 처리
- `@app.delete()`: DELETE 요청 처리

**동작 과정**:
1. 데코레이터가 함수를 FastAPI 앱에 등록
2. 지정된 경로로 요청이 오면 해당 함수 실행
3. 함수의 반환값을 자동으로 JSON으로 변환
4. 자동 문서화(/docs, /redoc)에 추가

아래 코드는 FastAPI 동작을 **간단히 시뮬레이션**한 것입니다:


In [ ]:
# FastAPI의 동작을 간단히 시뮬레이션
class FakeApp:
    """FastAPI app의 동작을 흉내낸 클래스"""
    
    def get(self, path: str):
        """@app.get() 데코레이터를 흉내
        
        Args:
            path: URL 경로 (예: '/users')
        
        Returns:
            decorator: 함수를 등록하는 데코레이터
        """
        def decorator(func):
            # 실제 FastAPI에서는 여기서 라우팅 테이블에 등록
            print(f"✅ 경로 '{path}'에 함수 '{func.__name__}' 등록!")
            return func  # 원래 함수를 그대로 반환
        return decorator

# FastAPI 앱 생성 (시뮬레이션)
app = FakeApp()

# FastAPI 스타일로 API 엔드포인트 정의
@app.get("/users")
def get_users():
    """사용자 목록을 반환하는 API"""
    return {"users": ["Alice", "Bob"]}

# 출력: ✅ 경로 '/users'에 함수 'get_users' 등록!

**실행 결과:**
```
✅ 경로 '/users'에 함수 'get_users' 등록!
```

**코드 해석:**
1. `FakeApp` 클래스는 FastAPI의 `app` 객체를 간단히 흉내냅니다
2. `get()` 메서드는 경로(path)를 받아서 데코레이터를 반환합니다
3. `@app.get("/users")`를 붙이면 해당 경로에 함수가 등록됩니다
4. 실제 FastAPI에서는 이렇게 등록된 함수들이 HTTP 요청을 처리합니다

### 🚀 실제 FastAPI에서는?

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/users")  # GET http://localhost:8000/users
def get_users():
    return {"users": ["Alice", "Bob"]}

@app.post("/users")  # POST http://localhost:8000/users
def create_user(name: str):
    return {"message": f"{name} 생성됨!"}
```

**FastAPI의 마법**:
- 데코레이터만 붙이면 → 즉시 REST API 완성!
- 타입 힌트 자동 검증
- 자동 문서 생성
- JSON 변환 자동화

### 💡 중요한 점

지금 당장 데코레이터를 **만들 필요는 없습니다**. FastAPI에서 제공하는 `@app.get()` 같은 데코레이터를 **사용하는 법**만 알면 충분합니다!

---

## 6. @property 데코레이터

### 메서드를 속성처럼 사용하기

`@property`는 Python의 **내장 데코레이터**로, 메서드를 마치 속성(변수)처럼 사용할 수 있게 해줍니다.

**일반 메서드 vs @property**:
```python
# 일반 메서드: 괄호() 필요
user.get_full_name()  # 메서드 호출

# @property: 괄호() 없이 속성처럼
user.full_name  # 속성처럼 접근!
```

**왜 사용할까요?**
- **가독성**: 코드가 더 자연스럽고 읽기 쉬움
- **캡슐화**: 내부 로직을 숨기고 간단한 인터페이스 제공
- **계산된 속성**: 매번 계산이 필요한 값을 속성처럼 제공


In [ ]:
class User:
    """@property 사용 예제"""
    
    def __init__(self, first_name: str, last_name: str):
        # private 변수 (관례적으로 _로 시작)
        self._first_name = first_name
        self._last_name = last_name
    
    @property
    def full_name(self) -> str:
        """전체 이름을 계산하여 반환
        
        @property 덕분에 메서드지만 속성처럼 사용 가능!
        user.full_name() ❌  괄호 없이!
        user.full_name ✅
        """
        return f"{self._first_name} {self._last_name}"
    
    @property
    def name(self) -> str:
        """이름만 반환"""
        return self._first_name

# User 객체 생성
user = User("홍", "길동")

# @property 덕분에 괄호 없이 속성처럼 접근!
print(user.full_name)  # 출력: 홍 길동 (메서드인데 () 없이!)
print(user.name)  # 출력: 홍

# 만약 @property가 없었다면?
# print(user.full_name())  # 이렇게 괄호를 붙여야 했을 것!

**실행 결과:**
```
홍 길동
홍
```

**코드 해석:**
1. `_first_name`, `_last_name`은 private 변수 (직접 접근 권장하지 않음)
2. `@property`를 붙이면 `full_name`을 메서드가 아닌 속성처럼 사용
3. `user.full_name`은 실제로는 메서드를 호출하지만, 사용자에게는 속성처럼 보임
4. 값이 필요할 때마다 계산되므로 항상 최신 상태 유지

### 💡 실무 활용 예시

```python
class Product:
    def __init__(self, price: int, tax_rate: float = 0.1):
        self.price = price
        self.tax_rate = tax_rate
    
    @property
    def total_price(self) -> float:
        """세금 포함 가격 계산 (매번 최신 값)"""
        return self.price * (1 + self.tax_rate)

product = Product(10000)
print(product.total_price)  # 11000.0 (속성처럼 접근!)

# 가격 변경
product.price = 20000
print(product.total_price)  # 22000.0 (자동으로 재계산!)
```

### ⚠️ 주의사항

- `@property`는 **읽기 전용**입니다 (setter를 추가하지 않으면)
- 복잡한 계산이 필요하면 캐싱 고려
- 너무 무거운 작업은 피하세요 (속성처럼 보이니까!)

---

## 📌 핵심 정리

이번 시간에 배운 내용:

### 1. 변수 스코프
- **지역 변수**: 함수 안에서만 존재, 독립적
- **전역 변수**: 프로그램 전체에서 사용, `global` 키워드로 수정
- **클래스 변수**: 모든 인스턴스가 공유 (`클래스명.변수`)
- **인스턴스 변수**: 각 객체마다 독립적 (`self.변수`)

### 2. 데코레이터 개념
- **데코레이터**: 함수를 꾸며서 기능을 추가하는 함수
- **@ 문법**: 데코레이터를 적용하는 간편한 방법
- **FastAPI 라우팅**: `@app.get("/users")` → URL과 함수 연결
- **@property**: 메서드를 속성처럼 사용 가능하게

### 3. 실무 연결
```python
# FastAPI에서 이렇게 사용됩니다!
from fastapi import FastAPI

app = FastAPI()

# 전역 설정
MAX_ITEMS = 100  # 전역 변수

@app.get("/users")  # 데코레이터로 라우팅
def get_users(limit: int = 10):
    if limit > MAX_ITEMS:  # 전역 변수 읽기
        limit = MAX_ITEMS
    return {"users": [...], "limit": limit}
```

---

## 다음 단계

**강사와 함께 실습**:
- `2_practice.py`: 스코프와 데코레이터 직접 실험하기
  - 전역 변수 활용 연습
  - 클래스 변수와 인스턴스 변수 구분
  - @property 활용해보기

**스스로 문제 풀기**:
- `3_exercise.py`: 방문자 카운터 시스템 구현
  - 전역 변수로 카운터 관리
  - 클래스 변수로 통계 추적

---

## 기억할 점

### ⭐ 중요한 개념
1. **함수 내부 변수는 지역 변수** (독립적, 안전함)
2. **전역 변수 수정 시 `global` 필수** (없으면 에러!)
3. **클래스 변수는 공유, 인스턴스 변수는 독립**
4. **데코레이터는 @ 문법으로 사용**
5. **FastAPI의 핵심은 라우팅 데코레이터** (`@app.get` 등)

### 💡 실무 팁
- 가능하면 **지역 변수 사용**을 우선하세요
- 전역 변수는 **설정값, 상수**로 제한
- 데코레이터를 **직접 만들 필요는 없어요** (사용법만 알면 OK!)
- FastAPI의 `@app.get()` 패턴에 익숙해지세요

### 🚀 다음 시간 예고
다음 시간에는 **import와 모듈**을 배웁니다!
- 다른 사람의 코드 가져오기
- 내 코드를 모듈로 만들기
- FastAPI 프로젝트 구조화

---

**수고하셨습니다! 🎉**

이제 `2_practice.py`로 넘어가서 강사와 함께 실습해봅시다!